In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
import pydicom as dicom

import os
import re
from numpy import nan as Nan

In [2]:
##################################### ADDITIONAL FUNCTIONS #####################################################

def strip_mpr_lad_name(mpr_name):
    """
    Strip MPR name of the LAD artery. We do this step because the name in the doctor's report 
    is not equal to the name in the MPR. 
    
    Returns:
        - str: striped string
    """
    return "".join(mpr_name.split()).replace('*', '').replace('original', '')

def read_and_strip_record(path_to_record):
    '''
    Read record file and remove empty rows and rows with all NaNs.
    
    Returns:
        - Pandas DataFrame: 
    '''
    excel_file = pd.read_excel(path_to_record,index_col=None, header=None)
    excel_file.dropna(how='all')
    excel_file.rename(columns={0: 'a', 1: 'b'}, inplace=True)
    excel_file = excel_file.fillna('  ')
    excel_file = excel_file.replace('', '  ', regex=True)
    excel_file = excel_file.drop(excel_file[excel_file['a'].str.isspace()].index)
    return excel_file

def get_lad_info_from_report(striped_record, artery_type):
    """
    Takes striped(without any empty lines and NaNs) and returns info only about the certain artery type. 
    
    Returns:
        - list: each element is the string with some info about certain artery type
    """
    lad_info = []
    wether_add = False
    lad_info.append(striped_record.iloc[0]['b'])
    for ind, row_value in striped_record.iterrows():
        if wether_add and row_value['a'].isupper():
            break
        if wether_add:
            lad_info.append(row_value['a'])
        
        if artery_type in row_value['a']:
            wether_add = True
    return lad_info


def get_level_of_stenosis_from_string(artery_info):
    """
    Returns:
        - list of str: each element is the string with percentage of stenosis. 
    """
    return [x.strip() for x in re.findall(r'.\d{1,3}.?\d{1,3}\%', artery_info)]

In [3]:
reports_path = r'/home/petryshak/CoronaryArteryPlaqueIdentification/data/reports'
list_of_files = os.listdir(reports_path)

In [4]:
# GET LIST OF ALL BRANCHES OF RCA
unique_rca = list()
for i in range(len(list_of_files)):
    try:
        cur_report_name = list_of_files[i]
        cur_file = read_and_strip_record(os.path.join(reports_path, cur_report_name))
        cur_patient_info = get_lad_info_from_report(cur_file, 'RIGHT CORONARY')
        cur_patient_info.pop(0)
        for name in [el.split(' -')[0] for el in cur_patient_info]:
            new_branch = name.lstrip()
            if len(new_branch) < 20 and '.' not in new_branch:
                unique_rca.append(new_branch)
    except:
        print('Error!')

In [5]:
# GET LIST OF ALL BRANCHES OF LEFT CIRCUMFLEX 
unique_lcx = list()
for i in range(len(list_of_files)):
    try:
        cur_report_name = list_of_files[i]
        cur_file = read_and_strip_record(os.path.join(reports_path, cur_report_name))
        cur_patient_info = get_lad_info_from_report(cur_file, 'LEFT CIRCUMFLEX')
        cur_patient_info.pop(0)
        for name in [el.split(' -')[0] for el in cur_patient_info]:
            new_branch = name.lstrip()
            if len(new_branch) < 20 and '.' not in new_branch:
                unique_lcx.append(new_branch)
    except:
        print('Error!')

In [6]:
# UNIQUE LCX SECTIONS in REPORTS
pd.Series(unique_lcx).value_counts()[pd.Series(unique_lcx).value_counts() > 20].index

Index(['Proximal', 'OM1', 'Distal', 'OM2', 'OM3', 'PDA', 'PLV', 'PL'], dtype='object')

In [7]:
# UNIQUE RCA SECTIONS in REPORTS
pd.Series(unique_rca).value_counts()[pd.Series(unique_rca).value_counts() > 5].index

Index(['Proximal', 'Distal', 'Mid', 'PDA', 'PLV'], dtype='object')

In [8]:
# UNIQUE BRANCHES in IMAGES
# unique_branches_images = pd.read_csv('num_unique_branches.csv')

In [14]:
# GET LCX TABLE

extracted_lcx_df = pd.DataFrame(columns=['PATIENT_ID', 'REPORT_ID', 'PROXIMAL', 'DISTAL', 'OM', 'OM1', 'OM2', 'OM3', 
                                     'PDA', 'PLV'])


def calculate_stenosis(info):
    score = get_level_of_stenosis_from_string(info)
    score =  score[0] if score else 'NORMAL'
    return score

for i in tqdm(range(len(list_of_files))):
    try:
        cur_report_name = list_of_files[i]
        cur_file = read_and_strip_record(os.path.join(reports_path, cur_report_name))
        cur_patient_info = get_lad_info_from_report(cur_file, 'LEFT CIRCUMFLEX')

        new_row = pd.Series(['-','-','-','-','-','-','-','-', '-','-'], index=extracted_lcx_df.columns)
        new_row['PATIENT_ID'] = cur_patient_info[0]
        new_row['REPORT_ID'] = cur_report_name.split('.')[0].split('RT ')[1]
        print(cur_report_name.split('.')[0].split('RT ')[1])
        cur_patient_info.pop(0)
        list_of_lcx_branches = [x + ' ' for x in list(extracted_lcx_df.columns)]
        for line_info in cur_patient_info:
            artery_area_name = [x for x in list_of_lcx_branches 
                                if x in line_info or x.lower() in line_info or x.title() in line_info]
            if len(artery_area_name) >=1:
                for area_name in artery_area_name:
                    area_name = area_name.strip()
                    stenosis_score = calculate_stenosis(line_info)
                    new_row.loc[area_name] = stenosis_score
            else:
                continue
        extracted_lcx_df = extracted_lcx_df.append(new_row, ignore_index=True)
    except:
        print("Error!")
extracted_lcx_df.head()

  2%|▏         | 16/744 [00:00<00:09, 74.98it/s]

CTCAHER21101967
CTCAGRH27071943
CTCADRG22021959
CTCAXUZ07071955
CTCAQUD02121959
CTCAOSP05031947
CTCAHIP19061975
CTCAMCF02061941
CTCAKAG20081953
CTCASHZ19081947
CTCAVIC18091963
CTCACUY07051952
CTCAAND19041968
CTCAGRP22061951
CTCABRV22041952
CTCASIK14121953


  4%|▍         | 31/744 [00:00<00:09, 72.13it/s]

1020 CTCA1963
CTCABUT30081955
CTCAROJ23091952
CTCAMOW12101970
CTCAABK06031973
1026 CTCA1953
CTCASTR17021954
CTCAFIM15041952
CTCACOD18031960
CTCABAD21091949
CTCAHUR24071959
CTCALID30101976
CTCAHAF13091949
CTCAWED27071959
1044 CTCA1967


  6%|▋         | 47/744 [00:00<00:09, 74.37it/s]

...

 99%|█████████▉| 735/744 [00:09<00:00, 75.89it/s]

1023 CTCA1960
CTCACES20071965
CTCABIB16101983
1012 CTCA1962
CTCAMIA30041964
CTCABRW29041967
CTCASPB03121931
CTCAJHH20081965
CTCALIB10031961
CTCABAM11031964
CTCASCL12011981
CTCADOT21121946
CTCALID04061955
CTCAWOM06061941
CTCAVRC07081978
CTCADAG07031973
CTCARIS09111963


100%|██████████| 744/744 [00:10<00:00, 74.08it/s]

CTCAMIM08101949
CTCAODJ27091949
CTCALAN04091968
1058 CTCA1956
CTCABOJ29051945
CTCAGRC24011965
CTCAGAR01101953


,PATIENT_ID,REPORT_ID,PROXIMAL,DISTAL,OM,OM1,OM2,OM3,PDA,PLV
0,CTCAHER21101967,CTCAHER21101967,NORMAL,-,-,NORMAL,-,-,-,-
1,CTCAGRH27071943,CTCAGRH27071943,NORMAL,NORMAL,-,NORMAL,NORMAL,-,NORMAL,-
2,CTCADRG22021959,CTCADRG22021959,NORMAL,NORMAL,-,NORMAL,-,-,-,-
3,CTCAXUZ07071955,CTCAXUZ07071955,NORMAL,NORMAL,-,NORMAL,NORMAL,-,-,-
4,CTCAQUD02121959,CTCAQUD02121959,NORMAL,NORMAL,-,NORMAL,NORMAL,NORMAL,-,-


In [12]:
extracted_lcx_df.to_excel('lcx_labels.xlsx', index=False)

In [13]:
pd.value_counts(extracted_lcx_df['PLV'])

-         692
NORMAL     51
>70%        1
Name: PLV, dtype: int64

In [14]:
# GET RCA TABLE
extracted_rca_df = pd.DataFrame(columns=['PATIENT_ID', 'REPORT_ID', 'PROXIMAL', 'DISTAL', 'MID',
                                     'PDA', 'PLV'])

def calculate_stenosis_rca(info, cur_report_name):
    if len(info) > 100:
        return '-'
    score = get_level_of_stenosis_from_string(info)
    score = score[0] if score else '-'
    if 'normal' in info.lower():
        score = 'NORMAL' 
    return score

for i, el in enumerate(list_of_files):
    try:
        cur_report_name = list_of_files[i]
        cur_file = read_and_strip_record(os.path.join(reports_path, cur_report_name))
        cur_patient_info = get_lad_info_from_report(cur_file, 'RIGHT CORONARY')
        
        new_row = pd.Series(['-','-','-','-','-','-','-'], index=extracted_rca_df.columns)
        new_row['PATIENT_ID'] = cur_patient_info[0]
        new_row['REPORT_ID'] = cur_report_name.split('.')[0].split('RT ')[1]

        cur_patient_info.pop(0)
        list_of_lcx_branches = [x + ' ' for x in list(extracted_rca_df.columns)]
        for line_info in cur_patient_info:
            artery_area_name = [x for x in list_of_lcx_branches 
                                if x in line_info or x.lower() in line_info or x.title() in line_info]
            if len(artery_area_name) >=1:
                for area_name in artery_area_name:
                    area_name = area_name.strip()
                    stenosis_score = calculate_stenosis_rca(line_info, cur_report_name)
                    new_row.loc[area_name] = stenosis_score
            else:
                continue
        extracted_rca_df = extracted_rca_df.append(new_row, ignore_index=True)
    except:
        print("Error, wrong file (not excel)!")
extracted_rca_df.head()

,PATIENT_ID,REPORT_ID,PROXIMAL,DISTAL,MID,PDA,PLV
0,CTCAHER21101967,CTCAHER21101967,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL
1,CTCAGRH27071943,CTCAGRH27071943,-,-,-,-,-
2,CTCADRG22021959,CTCADRG22021959,NORMAL,<25%,<25%,<25%,NORMAL
3,CTCAXUZ07071955,CTCAXUZ07071955,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL
4,CTCAQUD02121959,CTCAQUD02121959,NORMAL,NORMAL,70%,NORMAL,-


In [68]:
# extracted_rca_df.to_csv('rca_labels_titlesreports.csv', index=False)

In [15]:
extracted_rca_df.to_excel('rca_labelsreports.xlsx', index=False)

(744, 7)